# Query Caching

Query caching allows avoiding repeated queries to the database by caching the results locally for faster retrieval. 

To enable queries, set the query cache local path in `dj.config['query_cache']` and activate the query caching with  `conn.set_query_cache(query_cache)` where `conn` is the connection object.  

A reference to the connection object is kept as a property of the schema or table objects, e.g. `schema.connection`. Alternatively, the function `dj.conn()` returns the currently active connection object. 

The `query_cache` argument is an aribtrary string serving to differentiate cache states; setting a new value will effectively start a new cache, triggering retrieval of new values once.

Setting `query_cache=None` turns off query caching.

While query caching is enabled, any `insert` or `delete` calls and any transactions are disabled and will raise an error. This ensures that stale data are not used for updating the database in violation of data integrity.  

To clear the cache, delete the contents of the folder specified in `dj.config['query_cahce']`.


## A complete example

In [1]:
import datajoint as dj
import numpy as np
import os

Create a table to store image data and populate it

In [3]:
schema = dj.schema('test_query_caching')

@schema
class Image(dj.Manual):
    definition = """
    image_number : int
    ---
    image : longblob
    """

In [4]:
Image.insert1((1, np.random.randn(300,300)))

Time query without caching.

In [5]:
%%timeit
r = (Image & 'image_number=1').fetch1('image')

403 ms ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Enable query caching and note that queries are sped up.

In [6]:
dj.config['query_cache']=os.path.expanduser('~/tmp')
conn = dj.conn()
conn.set_query_cache('s0')

In [7]:
%%timeit
r = (Image & 'image_number=1').fetch1('image')

2.91 ms ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Any attempts to insert or delete data will result in an error while query caching is on.

In [8]:
Image.delete()

DataJointError: Only SELECT queries are allowed when query caching is on.

Turn off query caching. Data manua

In [9]:
# Turn off query caching
conn.set_query_cache(None)

In [10]:
Image.delete()

Deleting 1 rows from `test_query_caching`.`image`
Commit deletes? [yes, No]: yes
Deletes committed.
